## <center>As árveres somos nozes</center>

### <center>Manipulação de código via árvores sintáticas em Python</center>
<br><br>
<center>Python Brasil 2021</center>

## Quem sou eu
<img src="./profile-pic.png" style="float: right;" />

- Felipe Martins _(ele/dele)_
- Cearense morando no Recife
- Desenvolvedor senior, cerca de dez anos de experiência
- Apaixonado por Extreme Programming e Developer Experience
- [@ffmmjj_martins](https://twitter.com/ffmmjj_martins) 🌈🌈


Me chamem para um café, eu sou facinho ☕

https://eng.uber.com/piranha/

<img src="./piranha-blog.png" />

https://github.com/uber/piranha/blob/master/report.pdf

<img src="./piranha-paper.png" />

<img src="./piranha-workflow.png" />

1. Desenvolvedora publica código protegido por flag;
2. A flag é detectada como inoperante depois de um certo tempo;
3. Piranha cria um PR para a desenvolvedora removendo aquela flag;
4. Desenvolvedora aprova/rejeita o PR;

In [ ]:
#exemplo1.py
MINHA_FLAG = True


if MINHA_FLAG: # Checa estado da flag
    print('Usa nova funcionalidade')
else:
    print('Usa funcionalidade antiga')

print('Faz isso aqui sempre')

Seria transformada no código a seguir após remover a flag:

In [ ]:
print('Usa nova funcionalidade')

print('Faz isso aqui sempre')

**Como fazer esse tipo de otimização automaticamente?**

In [ ]:
codigo_simplificado = []

with open('exemplo1.py', 'r') as fp:
    encontrou_condicao_procurada = False
    dentro_do_else_a_ignorar = False
    
    for linha in fp:
        if linha.startswith('if MINHA_FLAG:'):
            encontrou_condicao_procurada = True
        elif encontrou_condicao_procurada and linha.startswith('else:'):
            dentro_do_else_a_ignorar = True
        elif dentro_do_else_a_ignorar and not linha.startswith('   '):
            dentro_do_else_a_ignorar = False
        elif not dentro_do_else_a_ignorar and not linha.startswith('MINHA_FLAG = True'):
            codigo_simplificado.append(linha)

print(''.join(codigo_simplificado))

In [ ]:
    print('Usa nova funcionalidade')
print('Faz isso aqui sempre')

Problemas:
- Complexidade do código;
- Não conserta indentações após a remoção de blocos de código;
- Não funciona com blocos aninhados (um if dentro de outro if, por exemplo);
- Precisa checar manualmente indentações para saber se estamos saindo de um bloco de código;
- Não lida corretamente com a constante caso ela seja importada com um alias;
- Etc...

***Como podemos fazer isso de uma forma melhor?***

### <center>Uma digressão: o processo de interpretação do CPython</center>

- Implementação canônica do Python: https://github.com/python/cpython
<img src="./cpython-github.png" />

- Etapas de processamento de scripts Python:
<img src="./ast-parsing.png" style="width: 949px; height: 194px;"/>

<img src="./ast-parsing.png" style="width: 949px; height: 194px;"/>

<img src="./cst-example1.png" style="float: right;" />
- A primeira etapa (Lexer) extrai cada elemento da gramática do Python (as tokens) e as relações entre esses elementos para gerar uma Árvore Sintática Concreta (a CST);

<img src="./ast-parsing.png" style="width: 949px; height: 194px;"/>

<img src="./ast-example1.png" style="float: right;" />
- Essa CST é então usada como entrada para a próxima etapa do Parser, que vai analisar a estrutura "bruta" do código Python representado por ela e construir uma Árvore Sintática Abstrata (a AST).

**A principal diferença entre essas duas árvores é que a AST mantém apenas partes necessárias para a execução do código (ela descarta comentários, por exemplo) e representa o código usando abstrações com mais semântica do que a CST.**

Essas árvores sintáticas costumam ser usadas em compiladores e interpretadores para, entre outras coisas, analisar e realizar transformações ao código antes de convertê-lo em código executável, como por exemplo identificar e aplicar otimizações a ele.

***E se convertêssemos a representação do código em nosso exemplo original para uma árvore sintática, realizássemos a transformação desejada nessa árvore e depois a convertêssemos de volta para código Python?***

<img src="./ast-header.png" />

Por sorte, temos o módulo _ast_ na biblioteca-padrão do Python: https://docs.python.org/3/library/ast.html

In [ ]:
import ast
import astor

with open('exemplo1.py', 'r') as fp:
    codigo = ''.join(fp.readlines())

exemplo1_ast = ast.parse(codigo, 'exemplo1.py')
print(astor.to_source(exemplo1_ast))

In [ ]:
MINHA_FLAG = True
if MINHA_FLAG:
    print('Usa nova funcionalidade')
else:
    print('Usa funcionalidade antiga')
print('Faz isso aqui sempre')

**Comentários e linhas em branco do código original desaparecem** ☹️☹️

Se quisermos manter elementos como comentários, espaçamentos e linhas em branco presentes no código original, precisaremos trabalhar com a árvore sintática ***concreta***.

O módulo *ast* não trabalha com árvores concretas (apenas abstratas), logo não poderemos usá-la para trabalhar com CSTs.

A própria documentação desse módulo, no entanto, recomenda o uso de uma biblioteca terceira chamada libCST para isso: https://github.com/Instagram/LibCST
<img src="./libcst-github.png" styles="align: right" />

Essa biblioteca permite obter uma árvore sintática concreta que represente um script Python e oferece um framework chamado **Codemod** para modificação de código a partir dessa árvore.

*Vamos ver como usar o Codemod da libCST para resolver nosso problema original de uma forma mais robusta.*

O Codemod permite o uso do padrão ***Visitor*** para percorrer a árvore sintática concreta e realizar alguma ação para cada tipo de nó encontrado.

Ele faz isso permitindo que possamos definir métodos que são executados quando **encontramos** um certo tipo de nó e quando **abandonamos** um certo tipo de nó enquanto percorremos a árvore sintática.

*Recomendo dar uma olhada nesse guia para relembrar os detalhes do padrão Visitor: https://refactoring.guru/pt-br/design-patterns/visitor*

In [25]:
import libcst as cst
from ast import literal_eval
from libcst.codemod import CodemodContext, VisitorBasedCodemodCommand


class IfDetectorCommand(VisitorBasedCodemodCommand):
    DESCRIPTION: str = 'Detecta entrada e saída de blocos if'
    
    def visit_If(self, node):
        print('Entrou no if:', node.test.value)
        return True # True se os nós filhos precisam ser visitados
    
    def visit_Else(self, node):
        print('Entrou no else')
        return True
    
    def leave_If(self, original_node, updated_node):
        print('Saiu do if:', updated_node.test.value)
        return updated_node # Retorna o mesmo nó sem modificações
    
    def leave_Else(self, original_node, updated_node):
        print('Saiu do else')
        return updated_node

Vamos ilustrar o Codemod com um exemplo bem simples: um Visitor que apenas detecta se está visitando nós correspondendo a ifs e elses e imprime uma mensagem ao entrar e sair deles.
Note que a terminologia da biblioteca chama esses visitors de _Comandos_.

In [ ]:
import unittest
from libcst.codemod import CodemodTest

class IfDetectorCommandTests(CodemodTest):
    TRANSFORM = IfDetectorCommand
    
    def test_basic_scenario(self):
        codigo = """\
            # Redundante
            CONSTANT_CONDITION = True
            if CONSTANT_CONDITION:
                print('is true')
            else:
                print('is false')
            
            print('continua')
            """
        self.assertCodemod(codigo, codigo)

unittest.main(argv=[''], verbosity=2, exit=False)

Entrou no if: CONSTANT_CONDITION

Entrou no else

Saiu do else

Saiu do if: CONSTANT_CONDITION

Aqui podemos usar o CodemodTest, que é fornecido pela própria biblioteca para poder testar comandos.

Basicamente, precisamos declarar na variável estática TRANSFORM qual é a classe que implementa nosso comando e, na asserção do teste, dizer qual é o código que será processado pelo comando e qual é o resultado esperado.

Como nosso comando não altera a árvore sintática do código, nenhuma modificação é feita nele.

### <center>Como usar então esses métodos do Visitor para alterar nós da árvore sintática?</center>

In [69]:
import libcst as cst
from ast import literal_eval
from libcst.codemod import CodemodContext, VisitorBasedCodemodCommand


class IfRemovalCommand(VisitorBasedCodemodCommand):
    DESCRIPTION: str = 'Remove ifs que usem a constante CONSTANT_CONDITION'
    
    def __init(self):
        self.dentro_do_if_a_ser_descartado = False
    
    def visit_If(self, node):
        if node.test.value == 'CONSTANT_CONDITION':
            self.dentro_do_if_a_ser_descartado = True
    
    def leave_If(self, original_node, updated_node):
        if self.dentro_do_if_a_ser_descartado:
            # Evita que outros ifs sejam afetados
            self.dentro_do_if_a_ser_descartado = False
            # Substitui o if pelo conteúdo do bloco dele
            return cst.FlattenSentinel(updated_node.body.body)

In [ ]:
class IfRemovalCommandTests(CodemodTest):
    TRANSFORM = IfRemovalCommand
    
    def test_basic_scenario(self):
        codigo = """\
            # Redundante
            CONSTANT_CONDITION = True
            if CONSTANT_CONDITION: 
                print('true')
            else:
                print('false')
                
            print('continua')
            """
        resultado = """\
            # Redundante
            CONSTANT_CONDITION = True
            print('true')
            
            print('continua')
            """
        self.assertCodemod(codigo, resultado)

unittest.main(argv=[''], verbosity=2, exit=False)

É possível usar seu Codemod customizado como um comando no terminal para aplicá-lo a um projeto inteiro:

In [83]:
# Inicializa o Codemod no diretório atual
!python3 -m libcst.tool initialize .

Successfully wrote default config file to /home/felipe/Documents/dev/pythonbr2021/.libcst.codemod.yaml


É necessário incluir no arquivo de configuração **.libcst.codemod.yaml** (criado no passo anterior) o nome do módulo onde está o nosso Codemod customizado:

In [84]:
!sed -i 's/libcst.codemod.commands/meuscodemods/g' .libcst.codemod.yaml
!cat .libcst.codemod.yaml

# String that LibCST should look for in code which indicates that the
# module is generated code.
generated_code_marker: '@generated'
# Command line and arguments for invoking a code formatter. Anything
# specified here must be capable of taking code via stdin and returning
# formatted code via stdout.
formatter: ['black', '-']
# List of regex patterns which LibCST will evaluate against filenames to
# determine if the module should be touched.
blacklist_patterns: []
# List of modules that contain codemods inside of them.
modules:
- 'meuscodemods'
# Absolute or relative path of the repository root, used for providing
# full-repo metadata. Relative paths should be specified with this file
# location as the base.
repo_root: '.'


In [85]:
# Arquivo que vamos modificar com nosso Codemod
!cat exemplos/codigo.py

# Redundante
CONSTANT_CONDITION = True
if CONSTANT_CONDITION: 
    print('true')
else:
    print('false')

print('continua')

In [11]:
# A opção -u faz com que apenas um diff seja gerado na saída-padrão
!python3 -m libcst.tool codemod -u -- codemod.IfRemovalCommand ./exemplos

Calculating full-repo metadata...
Executing codemod...
reformatted -
All done! ✨ 🍰 ✨
1 file reformatted.
--- /home/felipe/Documents/dev/pythonbr2021/exemplos/codigo.py
+++ /home/felipe/Documents/dev/pythonbr2021/exemplos/codigo.py
@@ -1,8 +1,6 @@
 # Redundante
 CONSTANT_CONDITION = True
-if CONSTANT_CONDITION: 
-    print('true')
-else:
-    print('false')
+print("true")
 
-print('continua')
+print("continua")
+
Finished codemodding 1 files!ng] estimated for 1 files to go...
 - Transformed 1 files successfully.
 - Skipped 0 files.
 - Failed to codemod 0 files.
 - 0 warnings were generated.


In [14]:
# Sem a opção -u, as alterações são aplicadas diretamente aos arquivos afetados
!python3 -m libcst.tool codemod -- codemod.IfRemovalCommand ./exemplos

Calculating full-repo metadata...
Executing codemod...
reformatted -
All done! ✨ 🍰 ✨
1 file reformatted.
Finished codemodding 1 files!ng] estimated for 1 files to go...
 - Transformed 1 files successfully.
 - Skipped 0 files.
 - Failed to codemod 0 files.
 - 0 warnings were generated.


In [15]:
!cat exemplos/codigo.py

# Redundante
CONSTANT_CONDITION = True
print("true")

print("continua")


Sucesso!

## Referências

- Documentação do libCST Codemod: https://libcst.readthedocs.io/en/latest/codemods.html
- Livro "CPython Internals": https://realpython.com/products/cpython-internals-book/
<img src="./cpython-internals-book.jpeg" />
- Padrão Visitor: https://refactoring.guru/pt-br/design-patterns/visitor

## <center> Obrigado! </center>